# PIA04 - Problema 2: Fallos de producto

Nota: no invento resultados. Donde falten ejecuciones, dejo placeholders para completar.

## Clonado de repositorio y carga de datos

**Objetivo y plan**
- Voy a clonar el repo de datasets si no existe.
- Busco y descomprimo datasets.zip.
- Localizo fallos_producto.csv sin rutas rigidas y lo cargo.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score, confusion_matrix, classification_report

# 1) Clonar repo si no existe
if not os.path.exists("PIA_04_datasets"):
    !git clone https://github.com/kachytronico/PIA_04_datasets

# 2) Buscar datasets.zip
!find . -name "datasets.zip"

# 3) Localizar datasets.zip con Python minimo
zip_path = None
for root, _, files in os.walk("."):
    if "datasets.zip" in files:
        zip_path = os.path.join(root, "datasets.zip")
        break
if zip_path is None:
    raise FileNotFoundError("datasets.zip no encontrado")

# 4) Descomprimir
!unzip -o "{zip_path}" -d "PIA_04_datasets/unzip"

# 5) Listar CSV
!find PIA_04_datasets/unzip -name "*.csv"

# 6) Cargar SOLO fallos_producto.csv
csv_path = None
for root, _, files in os.walk("PIA_04_datasets/unzip"):
    if "fallos_producto.csv" in files:
        csv_path = os.path.join(root, "fallos_producto.csv")
        break
if csv_path is None:
    raise FileNotFoundError("fallos_producto.csv no encontrado")

df_fallos = pd.read_csv(csv_path)
df_fallos.head()

**Conclusiones (a completar tras ejecutar)**
- Por completar: ruta detectada, shape y primeras filas visibles.

## Realiza un AED sobre el conjunto de datos.

**Objetivo y plan**
- Reviso tamanos, tipos y nulos generales.
- Confirmo que failure tiene NaN (unlabeled).
- Identifico columnas categoricas fijas.

In [ ]:
df_fallos.info()
display(df_fallos.head())
display(df_fallos.isna().mean().sort_values(ascending=False).head(10))

# Confirmo hechos fijos del dataset
print("Target:", "failure")
print("Unlabeled (NaN en failure):", df_fallos["failure"].isna().sum())
print("Categoricas esperadas:", ["product_code", "attribute_0", "attribute_1"])
print("Columna id presente:", "id" in df_fallos.columns)

**Conclusiones (a completar tras ejecutar)**
- Por completar: tamaño del dataset, % nulos, y confirmación de columnas clave.

## Estadísticos iniciales. 0.2 puntos

**Objetivo y plan**
- Obtengo estadisticos basicos para detectar rangos y posibles outliers.

In [ ]:
display(df_fallos.describe(include="all").T)

**Conclusiones (a completar tras ejecutar)**
- Por completar: rangos, medias y posibles valores extremos.

## Distribuciones de las variables numéricas del conjunto de datos. 0.3 puntos

**Objetivo y plan**
- Exploro forma de las distribuciones y posibles colas o asimetrias.

In [ ]:
num_df = df_fallos.select_dtypes(include="number")
num_df.hist(bins=30, figsize=(12, 8))
plt.suptitle("Distribuciones numericas")
plt.show()

**Conclusiones (a completar tras ejecutar)**
- Por completar: sesgos, colas u outliers visibles.

## Matriz de correlación. 0.5 puntos

**Objetivo y plan**
- Busco correlaciones altas para detectar redundancias.

In [ ]:
corr = num_df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Matriz de correlacion")
plt.show()

**Conclusiones (a completar tras ejecutar)**
- Por completar: correlaciones altas a considerar en el preprocesado.

## Realiza el preprocesamiento de datos de tu problema.

**Objetivo y plan**
- Defino columnas numericas y categoricas.
- Preparo el flujo de limpieza y transformaciones.

In [ ]:
cat_cols = ["product_code", "attribute_0", "attribute_1"]
num_cols = [c for c in df_fallos.columns if c not in cat_cols + ["failure", "id"]]
print("Categoricas:", cat_cols)
print("Numericas (sin id/failure):", num_cols)

**Conclusiones (a completar tras ejecutar)**
- Por completar: lista final de columnas y estrategia general.

## Reserva un conjunto de datos para validación y otro para testeo. 0.5 puntos

**Objetivo y plan**
- Separo labeled/unlabeled.
- Hago split estratificado solo dentro de labeled.
- Dejo valid/test intocables para self-training.

In [ ]:
df_labeled = df_fallos[df_fallos["failure"].notna()].copy()
df_unlabeled = df_fallos[df_fallos["failure"].isna()].copy()

X_labeled = df_labeled.drop(columns=["failure", "id"])
y_labeled = df_labeled["failure"].astype(int)

X_unlabeled = df_unlabeled.drop(columns=["failure", "id"])

X_train, X_temp, y_train, y_temp = train_test_split(
    X_labeled, y_labeled, test_size=0.30, stratify=y_labeled, random_state=42
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

print("Labeled:", df_labeled.shape, "Unlabeled:", df_unlabeled.shape)
print("Train/Valid/Test:", X_train.shape, X_valid.shape, X_test.shape)

**Conclusiones (a completar tras ejecutar)**
- Por completar: tamaños y proporciones de train/valid/test.

## Columnas inútiles, valores sin sentido y atípicos. 1 punto

**Objetivo y plan**
- Elimino `id` por ser identificador.
- Si aplico reglas de atipicos, las defino solo con train.

In [ ]:
# id ya fue eliminada en X_labeled y X_unlabeled
# Reglas de atipicos: si no hay criterio, no elimino por falta de dominio
print("Columnas en X_train:", X_train.columns.tolist())

**Conclusiones (a completar tras ejecutar)**
- Por completar: justificar eliminacion de `id` y decisiones sobre atipicos.

## Tratamiento de valores nulos. 0.5 puntos

**Objetivo y plan**
- Defino imputadores para numericas y categoricas.
- Integro OneHot y escalado en ColumnTransformer.

In [ ]:
num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ]
)

**Conclusiones (a completar tras ejecutar)**
- Por completar: porcentaje de nulos y metodo elegido.

## Análisis de variabilidad. 0.5 puntos

**Objetivo y plan**
- Analizo varianza SOLO en numericas de train.
- Elimino columnas con varianza casi cero si aparecen.

In [ ]:
low_var_cols = []
var_series = X_train[num_cols].var()
low_var_cols = var_series[var_series < 1e-6].index.tolist()

if low_var_cols:
    X_train = X_train.drop(columns=low_var_cols)
    X_valid = X_valid.drop(columns=low_var_cols)
    X_test = X_test.drop(columns=low_var_cols)
    X_unlabeled = X_unlabeled.drop(columns=low_var_cols)
    num_cols = [c for c in num_cols if c not in low_var_cols]

print("Low variance cols:", low_var_cols)

**Conclusiones (a completar tras ejecutar)**
- Por completar: columnas eliminadas por baja varianza o confirmacion de ninguna.

## Columnas categóricas. 0.5 punto

**Objetivo y plan**
- Ajusto OneHot SOLO con train.
- Transformo valid/test/unlabeled con el mismo objeto.

In [ ]:
X_train_proc = preprocessor.fit_transform(X_train)
X_valid_proc = preprocessor.transform(X_valid)
X_test_proc = preprocessor.transform(X_test)
X_unlabeled_proc = preprocessor.transform(X_unlabeled)

print("Shape train tras OneHot:", X_train_proc.shape)

**Conclusiones (a completar tras ejecutar)**
- Por completar: numero de columnas tras el encoding.

## Reducción de la dimensionalidad. 1 punto

**Objetivo y plan**
- Reduzco dimensionalidad con TruncatedSVD (apto para sparse).
- Ajusto SOLO con train y busco conservar ~0.90 de varianza.

In [ ]:
# Busco n_components que alcance ~0.90 de varianza con train
max_components = min(200, X_train_proc.shape[1] - 1)
if max_components < 2:
    max_components = 2

svd_probe = TruncatedSVD(n_components=max_components, random_state=42)
X_train_probe = svd_probe.fit_transform(X_train_proc)
cum_var = np.cumsum(svd_probe.explained_variance_ratio_)
n_components_90 = int(np.searchsorted(cum_var, 0.90) + 1)
n_components_90 = max(2, min(n_components_90, max_components))

svd = TruncatedSVD(n_components=n_components_90, random_state=42)
X_train_red = svd.fit_transform(X_train_proc)
X_valid_red = svd.transform(X_valid_proc)
X_test_red = svd.transform(X_test_proc)
X_unlabeled_red = svd.transform(X_unlabeled_proc)

print("n_components_90:", n_components_90)
print("Varianza explicada (train):", svd.explained_variance_ratio_.sum())

**Conclusiones (a completar tras ejecutar)**
- Por completar: componentes elegidos y varianza explicada.

## Realiza un etiquetado automático. 1 punto

**Objetivo y plan**
- Aplico self-training iterativo con threshold 0.90.
- Registro tabla por iteracion y grafico de remaining.

In [ ]:
base_model = LogisticRegression(max_iter=2000, class_weight="balanced")
threshold = 0.90
max_iters = 10

X_train_aug = X_train_red.copy()
y_train_aug = y_train.to_numpy().copy()
X_unl = X_unlabeled_red.copy()

history = []
for it in range(1, max_iters + 1):
    if X_unl.shape[0] == 0:
        history.append({"iter": it, "added": 0, "remaining": 0})
        break
    base_model.fit(X_train_aug, y_train_aug)
    proba = base_model.predict_proba(X_unl)
    max_proba = proba.max(axis=1)
    pseudo_mask = max_proba >= threshold
    added = int(pseudo_mask.sum())
    remaining = int(X_unl.shape[0] - added)
    history.append({"iter": it, "added": added, "remaining": remaining})
    if added == 0:
        break
    y_pseudo = proba[pseudo_mask].argmax(axis=1)
    X_train_aug = np.vstack([X_train_aug, X_unl[pseudo_mask]])
    y_train_aug = np.concatenate([y_train_aug, y_pseudo])
    X_unl = X_unl[~pseudo_mask]

history_df = pd.DataFrame(history)
display(history_df)

plt.figure(figsize=(6, 4))
plt.plot(history_df["iter"], history_df["remaining"], marker="o")
plt.title("Unlabeled restantes por iteracion")
plt.xlabel("iter")
plt.ylabel("remaining")
plt.grid(True)
plt.show()

**Conclusiones (a completar tras ejecutar)**
- Por completar: filas pseudo-etiquetadas y criterio de parada.

## Entrena y optimiza distintos modelos supervisados.

**Objetivo y plan**
- Entreno 3 modelos distintos con train_aug.
- Optimizo con CV interno y evalúo en valid.
- Reporto accuracy y f1/balanced_accuracy.

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model_results = {}

**Conclusiones (a completar tras ejecutar)**
- Por completar: comparar resultados en valid.

## Modelo 1. 1 punto

**Objetivo y plan**
- LogisticRegression con GridSearchCV.
- Evalúo en valid con accuracy y f1.

In [ ]:
lr = LogisticRegression(max_iter=3000, class_weight="balanced")
lr_grid = {
    "C": [0.1, 1.0, 10.0],
    "solver": ["lbfgs"],
}
lr_search = GridSearchCV(lr, lr_grid, cv=cv, scoring="balanced_accuracy", n_jobs=-1)
lr_search.fit(X_train_aug, y_train_aug)

lr_best = lr_search.best_estimator_
valid_pred_lr = lr_best.predict(X_valid_red)

model_results["model_1"] = {
    "name": "LogisticRegression",
    "estimator": lr_best,
    "valid_accuracy": accuracy_score(y_valid, valid_pred_lr),
    "valid_f1": f1_score(y_valid, valid_pred_lr),
    "valid_bal_acc": balanced_accuracy_score(y_valid, valid_pred_lr),
    "best_params": lr_search.best_params_,
}
model_results["model_1"]

**Conclusiones (a completar tras ejecutar)**
- Por completar: mejores parametros y metricas de valid del modelo 1.

## Modelo 2. 1 punto

**Objetivo y plan**
- RandomForest con RandomizedSearchCV.
- Evalúo en valid con accuracy y f1.

In [ ]:
rf = RandomForestClassifier(random_state=42, class_weight="balanced")
rf_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 5, 10],
    "min_samples_split": [2, 5, 10],
}
rf_search = RandomizedSearchCV(rf, rf_grid, n_iter=6, cv=cv, scoring="balanced_accuracy", n_jobs=-1, random_state=42)
rf_search.fit(X_train_aug, y_train_aug)

rf_best = rf_search.best_estimator_
valid_pred_rf = rf_best.predict(X_valid_red)

model_results["model_2"] = {
    "name": "RandomForest",
    "estimator": rf_best,
    "valid_accuracy": accuracy_score(y_valid, valid_pred_rf),
    "valid_f1": f1_score(y_valid, valid_pred_rf),
    "valid_bal_acc": balanced_accuracy_score(y_valid, valid_pred_rf),
    "best_params": rf_search.best_params_,
}
model_results["model_2"]

**Conclusiones (a completar tras ejecutar)**
- Por completar: mejores parametros y metricas de valid del modelo 2.

## Modelo 3. 1 punto

**Objetivo y plan**
- SVC con GridSearchCV y probability=True.
- Evalúo en valid con accuracy y f1.

In [ ]:
svc = SVC(probability=True, class_weight="balanced")
svc_grid = {
    "C": [0.5, 1.0, 2.0],
    "gamma": ["scale", "auto"],
    "kernel": ["rbf"],
}
svc_search = GridSearchCV(svc, svc_grid, cv=cv, scoring="balanced_accuracy", n_jobs=-1)
svc_search.fit(X_train_aug, y_train_aug)

svc_best = svc_search.best_estimator_
valid_pred_svc = svc_best.predict(X_valid_red)

model_results["model_3"] = {
    "name": "SVC",
    "estimator": svc_best,
    "valid_accuracy": accuracy_score(y_valid, valid_pred_svc),
    "valid_f1": f1_score(y_valid, valid_pred_svc),
    "valid_bal_acc": balanced_accuracy_score(y_valid, valid_pred_svc),
    "best_params": svc_search.best_params_,
}
model_results["model_3"]

**Conclusiones (a completar tras ejecutar)**
- Por completar: mejores parametros y metricas de valid del modelo 3.

## Crea un modelo ensemble y explica el criterio que utilizas. 1 punto

**Objetivo y plan**
- Hago soft voting con pesos proporcionales al rendimiento en valid.
- Comparo ensemble vs individuales en valid.

In [ ]:
weights = [
    model_results["model_1"]["valid_bal_acc"],
    model_results["model_2"]["valid_bal_acc"],
    model_results["model_3"]["valid_bal_acc"],
]

ensemble = VotingClassifier(
    estimators=[
        ("lr", model_results["model_1"]["estimator"]),
        ("rf", model_results["model_2"]["estimator"]),
        ("svc", model_results["model_3"]["estimator"]),
    ],
    voting="soft",
    weights=weights,
)
ensemble.fit(X_train_aug, y_train_aug)

valid_pred_ens = ensemble.predict(X_valid_red)
print("Valid accuracy:", accuracy_score(y_valid, valid_pred_ens))
print("Valid f1:", f1_score(y_valid, valid_pred_ens))
print("Valid balanced_accuracy:", balanced_accuracy_score(y_valid, valid_pred_ens))

**Conclusiones (a completar tras ejecutar)**
- Por completar: criterio de pesos y mejora frente a modelos individuales.

## Evaluación final (solo test_labeled)

**Objetivo y plan**
- Evalúo una sola vez con test_labeled.
- Reporto matriz de confusión y classification report.

In [ ]:
test_pred = ensemble.predict(X_test_red)
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Balanced accuracy:", balanced_accuracy_score(y_test, test_pred))
print("F1:", f1_score(y_test, test_pred))

cm = confusion_matrix(y_test, test_pred)
print("Confusion matrix:\n", cm)
print(classification_report(y_test, test_pred))

**Conclusiones (a completar tras ejecutar)**
- Por completar: métricas finales y observaciones.
- Si accuracy ~0.55, concluyo que hace falta más etiquetado manual.

## Asserts anti-leakage

In [ ]:
assert y_valid.notna().all() and y_test.notna().all()
assert df_unlabeled["failure"].isna().all()
assert set(X_train.index).isdisjoint(set(X_valid.index))
assert set(X_train.index).isdisjoint(set(X_test.index))
assert set(X_valid.index).isdisjoint(set(X_test.index))
assert df_fallos.loc[X_valid.index, "failure"].notna().all()
assert df_fallos.loc[X_test.index, "failure"].notna().all()